In [87]:
import numpy as np
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
from cobra.util import create_stoichiometric_matrix


In [88]:
model = read_sbml_model("e_coli_core.xml")
medium = model.medium
del(medium["EX_co2_e"])
model.medium = medium

In [94]:
solution = model.optimize()
fum_flux = solution.fluxes['FUM']
print(fum_flux)

5.064375661482087


In [95]:
for reaction in model.reactions.query('FUM', 'id'):
    print(reaction)

FUM: fum_c + h2o_c <=> mal__L_c
FUMt2_2: fum_e + 2.0 h_e --> fum_c + 2.0 h_c


In [89]:
with model:
    model.objective = model.reactions.EX_ac_e
    print(model.optimize().objective_value)

0.0


In [90]:
for reaction in model.reactions.query('EX', 'id'):
    print(reaction.id, reaction.flux)

EX_ac_e 1.8644444444444446
EX_acald_e 0.0
EX_akg_e 0.0
EX_co2_e 1.8644444444444446
EX_etoh_e 0.0
EX_for_e 0.0
EX_fru_e 0.0
EX_fum_e 0.0
EX_glc__D_e -0.9322222222222223
EX_gln__L_e 0.0
EX_glu__L_e 0.0
EX_h_e 1.864444444444444
EX_h2o_e 1.8644444444444448
EX_lac__D_e 0.0
EX_mal__L_e 0.0
EX_nh4_e 0.0
EX_o2_e -1.864444444444445
EX_pi_e 0.0
EX_pyr_e 0.0
EX_succ_e 0.0


In [53]:
for reaction in model.reactions.query('fum', 'id'):
    print(reaction)

EX_fum_e: fum_e --> 


In [73]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,4.765,0,0.00%
o2_e,EX_o2_e,21.8,0,0.00%
pi_e,EX_pi_e,3.215,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-22.81,1,100.00%
h2o_e,EX_h2o_e,-29.18,0,0.00%
h_e,EX_h_e,-17.53,0,0.00%


In [55]:
print(model.objective)

Maximize
1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5


In [56]:
model.optimize().objective_value

0.8739215069684307

In [59]:
model.reactions.BIOMASS_Ecoli_core_w_GAM.flux

0.8739215069684307

In [60]:
model.reactions.EX_ac_e.flux

0.0

In [77]:
for reaction in model.reactions.query('EX', 'id'):
    print(reaction.id, reaction.flux)

EX_ac_e -1.2308594016704104e-13
EX_acald_e 0.0
EX_akg_e 0.0
EX_co2_e 22.80983331020509
EX_etoh_e 0.0
EX_for_e 0.0
EX_fru_e 0.0
EX_fum_e 0.0
EX_glc__D_e -10.0
EX_gln__L_e 0.0
EX_glu__L_e 0.0
EX_h_e 17.530865429786573
EX_h2o_e 29.17582713556592
EX_lac__D_e 0.0
EX_mal__L_e 0.0
EX_nh4_e -4.765319193197471
EX_o2_e -21.799492655998872
EX_pi_e -3.2148950476847706
EX_pyr_e 0.0
EX_succ_e 0.0


In [64]:
optimal_growth = model.optimize().objective_value


In [66]:
with model:
    model.objective = model.reactions.EX_ac_e
    solution = model.optimize()
    optimal_growth = solution.objective_value
    print(optimal_growth)
    print(model.reactions.EX_ac_e.flux)
    print(model.reactions.BIOMASS_Ecoli_core_w_GAM.flux)
    model.reactions.BIOMASS_Ecoli_core_w_GAM.lower_bound = optimal_growth
    print(model.reactions.EX_ac_e.flux)
    print(model.reactions.BIOMASS_Ecoli_core_w_GAM.flux)

20.00000000000018
20.00000000000018
0.0
20.00000000000018
0.0


In [72]:
optimal_growth = model.optimize().objective_value
with model:
    model.objective = model.reactions.EX_fum_e
    model.reactions.BIOMASS_Ecoli_core_w_GAM.lower_bound = optimal_growth*0.2
    production_FUM_optimal_growth = model.optimize().objective_value
    print(production_FUM_optimal_growth)


0.0


In [21]:
FUM_ref_flux = solution.fluxes.EX_fum_e
FUM_ref_flux

0.0

In [23]:
solution.fluxes[solution.fluxes != 0.]

ACONTa                      6.007250e+00
ACONTb                      6.007250e+00
AKGDH                       5.064376e+00
ATPM                        8.390000e+00
ATPS4r                      4.551401e+01
BIOMASS_Ecoli_core_w_GAM    8.739215e-01
CO2t                       -2.280983e+01
CS                          6.007250e+00
CYTBD                       4.359899e+01
ENO                         1.471614e+01
EX_co2_e                    2.280983e+01
EX_glc__D_e                -1.000000e+01
EX_h_e                      1.753087e+01
EX_h2o_e                    2.917583e+01
EX_nh4_e                   -4.765319e+00
EX_o2_e                    -2.179949e+01
EX_pi_e                    -3.214895e+00
FBA                         7.477382e+00
FUM                         5.064376e+00
G6PDH2r                     4.959985e+00
GAPD                        1.602353e+01
GLCpts                      1.000000e+01
GLNS                        2.234617e-01
GLNabc                      2.722410e-16
GLUDy           

In [149]:
model = read_sbml_model("e_coli_core.xml")
medium = model.medium
del(medium["EX_co2_e"])
model.medium = medium

In [150]:
solution = model.optimize()
max_growth = model.reactions.BIOMASS_Ecoli_core_w_GAM.flux
max_growth

0.8739215069684307

In [194]:
glc_essentiality = set()
for gene in model.genes:
    with model:
        gene.knock_out()
        mutant_growth = model.slim_optimize(error_value=0.)
        if  mutant_growth <  0.05:
            glc_essentiality.add(gene.id)
print(glc_essentiality)

{'b2779', 'b1136', 'b2926', 'b2416', 'b0720', 'b1779', 'b2415'}


In [195]:
len(glc_essentiality)

7

In [196]:
glc_essentiality

{'b0720', 'b1136', 'b1779', 'b2415', 'b2416', 'b2779', 'b2926'}

In [207]:
ac_essentiality = set()
medium = model.medium
with model:
    medium["EX_glc__D_e"] = 0
    medium["EX_ac_e"] = 10
    model.medium = medium
    for gene in model.genes:
        with model:
            gene.knock_out()
            mutant_growth = model.slim_optimize(error_value=0.)
            if  mutant_growth <  0.05:
                ac_essentiality.add(gene.id)

In [208]:
len(ac_essentiality)

34

In [209]:
ac_essentiality

{'b0720',
 'b0721',
 'b0722',
 'b0723',
 'b0724',
 'b1136',
 'b1779',
 'b2276',
 'b2277',
 'b2278',
 'b2279',
 'b2280',
 'b2281',
 'b2282',
 'b2283',
 'b2284',
 'b2285',
 'b2286',
 'b2287',
 'b2288',
 'b2779',
 'b2926',
 'b3731',
 'b3732',
 'b3733',
 'b3734',
 'b3735',
 'b3736',
 'b3737',
 'b3738',
 'b3919',
 'b4015',
 'b4025',
 's0001'}

In [212]:
difference = ac_essentiality.difference(glc_essentiality) 
difference

{'b0721',
 'b0722',
 'b0723',
 'b0724',
 'b2276',
 'b2277',
 'b2278',
 'b2279',
 'b2280',
 'b2281',
 'b2282',
 'b2283',
 'b2284',
 'b2285',
 'b2286',
 'b2287',
 'b2288',
 'b3731',
 'b3732',
 'b3733',
 'b3734',
 'b3735',
 'b3736',
 'b3737',
 'b3738',
 'b3919',
 'b4015',
 'b4025',
 's0001'}